##### 1. nn.Block.name_scope(): 为每一个层或参数添加在系统中独一无二的name(prefix+"dense")
####  对forward()自动生成backward()

In [1]:
from mxnet.gluon import nn
class net(nn.Block):
    def __init__(self,**kw):
        super(net,self).__init__(**kw)
        with self.name_scope():
            self.dense0=nn.Dense(256)
            self.dense1=nn.Dense(10)
    def forward(self,x):
        print("X.shape: ")
        print(f"before:{x.shape}")
        x=self.dense0(x);
        x=nd.relu(x)
        print(f"After dense0:{x.shape}")
        x=self.dense1(x)
        print(f"After dense1: {x.shape}")
        return x
myNet=net(prefix="init_demo_")
from mxnet import nd 
x=nd.uniform(shape=(5,4,2,5))
myNet.initialize()
y=myNet(x)
y,myNet.dense0.name

X.shape: 
before:(5, 4, 2, 5)
After dense0:(5, 256)
After dense1: (5, 10)


(
 [[ 0.03207505 -0.13214687 -0.01917468 -0.10339475 -0.09550501  0.09729765
   -0.03277578  0.00927344  0.05275003 -0.02885164]
  [ 0.05110409 -0.03583925  0.02581121 -0.06956408 -0.00596018  0.158152
    0.05687293  0.00089315  0.0639057  -0.05084788]
  [ 0.05193184 -0.03240399 -0.02085465 -0.10293196 -0.05543653  0.11844829
   -0.01310616 -0.02696566  0.07564146 -0.03185817]
  [ 0.05791519 -0.09516785 -0.00303951 -0.09192937 -0.06963897  0.03071369
   -0.0009858   0.03261724  0.02710678 -0.05461043]
  [-0.00385509 -0.07300236  0.01278732 -0.09285736 -0.04360339  0.0752065
    0.0281924  -0.04088405 -0.01842174 -0.03803179]]
 <NDArray 5x10 @cpu(0)>,
 'init_demo_dense0')

In [2]:
class FancyMLP(nn.Block):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        with self.name_scope():
            self.dense=nn.Dense(100)
            self.weight=nd.uniform(shape=(100,20))
    def forward(self,x):
        x=self.dense(x)
        print(f"shape1: {x.shape}")
        x=nd.relu(nd.dot(x,self.weight)*2)
        print(f"shape2: {x.shape}")
        x=nd.relu(self.dense(x))
        print(f"shape3:{x.shape}")
        return x 
net=FancyMLP()
net.initialize()
x=nd.uniform(shape=(4,20))
net(x)

shape1: (4, 100)
shape2: (4, 20)
shape3:(4, 100)



[[0.05231703 0.         0.         0.         0.         0.05300057
  0.02378595 0.         0.         0.         0.         0.
  0.04138212 0.         0.         0.         0.01784178 0.01572579
  0.00923112 0.01330817 0.01812916 0.02996658 0.03407922 0.
  0.         0.         0.03785183 0.02372986 0.         0.
  0.01300334 0.         0.00033825 0.         0.01112824 0.
  0.00654261 0.0634444  0.01741542 0.00497361 0.         0.02569418
  0.         0.0182972  0.01540323 0.         0.01723471 0.03689558
  0.04422717 0.02280479 0.         0.01708727 0.00043204 0.00568333
  0.         0.         0.03839077 0.02157358 0.030027   0.
  0.         0.01673079 0.03767619 0.         0.         0.0351254
  0.02244431 0.         0.         0.04950608 0.01236251 0.
  0.00467286 0.01453074 0.         0.         0.         0.01396383
  0.04752699 0.         0.         0.         0.         0.02569895
  0.         0.03958415 0.         0.00114011 0.01651945 0.
  0.         0.00560093 0.00267281 0

In [3]:
x1=nd.random.uniform(shape=(3,3),dtype='float32')
x2=nd.random_uniform(shape=(3,3),dtype='float32')
x=nd.stack(x1,x2,axis=1)
k=nd.softmax(x)
k,k.argmax(axis=1)

(
 [[[0.35539865 0.34818497 0.29641646]
   [0.370534   0.3646386  0.2648274 ]]
 
  [[0.23788568 0.44341287 0.3187014 ]
   [0.41524166 0.3728222  0.21193618]]
 
  [[0.2471644  0.4737605  0.27907503]
   [0.43638325 0.23499651 0.32862025]]]
 <NDArray 3x2x3 @cpu(0)>,
 
 [[1. 1. 0.]
  [1. 0. 0.]
  [1. 0. 1.]]
 <NDArray 3x3 @cpu(0)>)

# 嵌套Block

In [4]:
class RecMLP(nn.Block):
    def __init__(self,**kwargs):
        super(RecMLP,self).__init__(**kwargs)
        self.net=nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(128,activation='relu'),
                        nn.Dense(20,activation='relu'))
            self.dense1=nn.Dense(10)
    def forward(self,x):
        return nd.relu(self.dense1(self.net(x)))
myNet=RecMLP(prefix="RecMLP_")
net=nn.Sequential()
net.add(myNet,
       nn.Dense(10))
net.initialize()
net

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(None -> 128, Activation(relu))
      (1): Dense(None -> 20, Activation(relu))
    )
    (dense1): Dense(None -> 10, linear)
  )
  (1): Dense(None -> 10, linear)
)

### initialize()

In [5]:
net=nn.Sequential()
net.add(nn.Dense(4),nn.Dense(2))
x=nd.random_normal(shape=(3,5))
net.initialize()
k=net(x)
layer1=net[0]
layer1.name,layer1.weight,layer1.bias,layer1.weight.grad(),k.shape

('dense1',
 Parameter dense1_weight (shape=(4, 5), dtype=float32),
 Parameter dense1_bias (shape=(4,), dtype=float32),
 
 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]
 <NDArray 4x5 @cpu(0)>,
 (3, 2))

### collect_params()

In [6]:
params=net.collect_params()
params,params['dense50_weight'].data()

KeyError: 'dense50_weight'

### 使用不同的初始化函数

In [16]:
from mxnet import init
# 参数来自于 均值为0，方差为0.02的正态分布
params.initialize(init=init.Normal(sigma=0.02),force_reinit=True)
params.initialize(init=init.One(),force_reinit=True)
layer1=net[0]
layer1.name,layer1.weight,layer1.weight.data()

('dense21',
 Parameter dense21_weight (shape=(4, 5), dtype=float32),
 
 [[ 0.00243308 -0.03123487 -0.05151046 -0.04551276  0.03036036]
  [ 0.0286264  -0.01455164 -0.00515897  0.00915898  0.04765999]
  [-0.04434082 -0.04131879 -0.04972132 -0.04690576 -0.00167212]
  [-0.05252337 -0.02021422  0.03109129  0.06166048 -0.06573658]]
 <NDArray 4x5 @cpu(0)>)

In [39]:
class myInit(init.Initializer):
    def __init__(self):
        super(myInit, self).__init__()
        self._verbose = True
    def _init_weight(self,_,arr):
        print("init weight: ",arr.shape)
        nd.random.uniform(low=1,high=10,out=arr)
    def _init_bias(self,_,arr):
        print("init bias: ",arr.shape)
        arr[:]=3
params.initialize(init=myInit(),force_reinit=True)
layer1=net[0]
layer1.collect_params(),layer1.weight.data(),layer1.bias.data() 

init weight:  (4, 5)
init weight:  (2, 4)


RuntimeError: Parameter 'dense80_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [ ]:
from mxnet import gluon
gluon.__file__


# 共享模型函数

In [40]:
net=nn.Sequential() 
net.add(nn.Dense(4,in_units=5,activation='relu'))
net.add(nn.Dense(4,in_units=5,activation='relu',params=net[-1].params),
       nn.Dense(3))
net.initialize(init=myInit()) 
p=nd.random_uniform(shape=(5,5))
net[0].weight.data(),net[0].bias.data() ,net[1].weight.data() 

init weight:  (4, 5)


(
 [[5.777741  6.4567733 1.7670976 7.3289576 6.9910994]
  [6.7130766 1.2667133 9.63228   9.003586  1.9296834]
  [7.1390705 8.804504  9.021829  1.2627121 1.0108682]
  [5.814252  9.43497   4.6381927 9.624766  5.7176547]]
 <NDArray 4x5 @cpu(0)>,
 
 [0. 0. 0. 0.]
 <NDArray 4 @cpu(0)>,
 
 [[5.777741  6.4567733 1.7670976 7.3289576 6.9910994]
  [6.7130766 1.2667133 9.63228   9.003586  1.9296834]
  [7.1390705 8.804504  9.021829  1.2627121 1.0108682]
  [5.814252  9.43497   4.6381927 9.624766  5.7176547]]
 <NDArray 4x5 @cpu(0)>)

# 序列化参数

In [46]:
x=nd.random_uniform(shape=(3,4))
y=nd.ones(shape=(4,))
filename="./para.params"
nd.save(filename,[x,y])
a=nd.load(filename)
a 

[
 [[0.5243798  0.8518537  0.1173803  0.3955438 ]
  [0.1598453  0.23307782 0.04680635 0.58543205]
  [0.97073144 0.74683636 0.00386035 0.60898864]]
 <NDArray 3x4 @cpu(0)>,
 
 [1. 1. 1. 1.]
 <NDArray 4 @cpu(0)>]

## 保存模型参数，下次直接读取,同样的输入会得到同样的输出，继续训练，不用像前面一样随机初始化，

In [75]:
from utils.try_gpu import try_gpu
net=nn.Sequential() 
net.add(nn.Dense(5,activation='relu'))
net.add(nn.Dropout(.5))
net.add(nn.Dense(3))
net.initialize() 
p=nd.random_uniform(shape=(6,3))
print("before: ",net(p))
net.save_parameters(filename)
net.load_parameters(filename,ctx=try_gpu())
net(p)

before:  
[[ 9.0064605e-05  3.9931675e-04 -8.1112765e-04]
 [ 1.3489560e-03  3.4629425e-04  2.0301142e-03]
 [ 7.4891583e-04 -1.6851187e-03  2.4013307e-03]
 [ 9.0796506e-04  6.8994763e-04  2.1678454e-04]
 [ 1.2787307e-03  1.3473642e-04  7.6159585e-04]
 [ 1.4077565e-03  2.3968915e-04  2.7096032e-03]]
<NDArray 6x3 @cpu(0)>


ValueError: data needs to either be a NDArray, dict of str, NDArray pairs or a list of NDarrays.

# 自定层

In [85]:
class CenteredLayer(nn.Block):
    def __init__(self,**kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self,x):
        return x/x.sum(axis=1,keepdims=True)
p=nd.random_uniform(1,5,shape=(2,2))
net=CenteredLayer()
# net.initialize() 
net(p)


[[2.3217623 4.5750914]
 [3.5535605 3.300247 ]]
<NDArray 2x2 @cpu(0)>

[[6.8968534]
 [6.8538074]]
<NDArray 2x1 @cpu(0)>



[[0.3366408  0.6633592 ]
 [0.51847976 0.48152024]]
<NDArray 2x2 @cpu(0)>

## `CenteredLayer`展示自定义层大概的样子，但缺少可以学习的模型参数

dense.weight.data(),weight是一个`Parameter `类型

In [91]:
net=nn.Sequential()
net.add(nn.Dense(12,activation='relu'),
       nn.Dense(6),CenteredLayer())
net.initialize()
# net(p)
net.collect_params()

sequential56_ (
  Parameter dense145_weight (shape=(12, 0), dtype=float32)
  Parameter dense145_bias (shape=(12,), dtype=float32)
  Parameter dense146_weight (shape=(6, 0), dtype=float32)
  Parameter dense146_bias (shape=(6,), dtype=float32)
)

In [97]:
from mxnet.gluon import Parameter 
params=Parameter("a_fancy_params",shape=(2,2))
params.initialize() 
params,params.data(),params.grad()

(Parameter a_fancy_params (shape=(2, 2), dtype=<class 'numpy.float32'>),
 
 [[-0.02985967  0.0388329 ]
  [-0.06727526 -0.049906  ]]
 <NDArray 2x2 @cpu(0)>)

自定义的层，这里指定了输入的大小，gluon的Dense不需要，这样的层涉及到hybridize

In [117]:
class myDense(nn.Block):
    def __init__(self,units,in_units,**kwargs):
        super(myDense,self).__init__(**kwargs)
        with self.name_scope():
            self.weight=self.params.get('weight',shape=(in_units,units))
            self.bias=self.params.get('bias',shape=(units,))
    def forward(self,x):
        print(x,self.weight.data())
        return nd.relu(nd.dot(x,self.weight.data())+self.bias.data())

net=nn.Sequential() 
net.add(myDense(12,in_units=6),
       myDense(4,in_units=12))
net.initialize() 
net(nd.uniform(shape=(20,6)))


[[0.305267   0.50176764 0.94306225 0.98064893 0.99065167 0.42098534]
 [0.19889222 0.58562815 0.65683836 0.5911709  0.10649532 0.05699885]
 [0.650914   0.72113764 0.82731324 0.77642107 0.68449855 0.7337627 ]
 [0.41733316 0.14674284 0.38306636 0.45388022 0.3931224  0.39715967]
 [0.58971184 0.38575968 0.8815673  0.06829276 0.9290662  0.7903047 ]
 [0.05352962 0.00492913 0.1816224  0.16326366 0.11222432 0.69682735]
 [0.19333464 0.22743139 0.3466078  0.84701735 0.50653166 0.6016257 ]
 [0.6294612  0.7532607  0.7321422  0.22580884 0.89011157 0.751381  ]
 [0.9890884  0.83605313 0.66285646 0.54201734 0.8453645  0.02740073]
 [0.77803886 0.3299729  0.30753204 0.5911424  0.87569225 0.9380934 ]
 [0.70351696 0.759252   0.6102408  0.4063043  0.79961526 0.70433503]
 [0.03457122 0.5853592  0.77023876 0.9695645  0.7317286  0.30387843]
 [0.2596984  0.3137554  0.2570693  0.22646162 0.6323033  0.67233455]
 [0.34529746 0.77892226 0.79658866 0.5126617  0.44614622 0.61526495]
 [0.7827494  0.04566117 0.9904718


[[0.06554423 0.         0.         0.        ]
 [0.06505423 0.         0.         0.        ]
 [0.06884457 0.         0.         0.        ]
 [0.06510217 0.         0.         0.        ]
 [0.06356238 0.         0.         0.        ]
 [0.06539616 0.         0.         0.        ]
 [0.06809764 0.         0.         0.        ]
 [0.06511023 0.         0.         0.        ]
 [0.06383993 0.         0.         0.        ]
 [0.06888574 0.         0.         0.        ]
 [0.06655781 0.         0.         0.        ]
 [0.06581445 0.         0.         0.        ]
 [0.0645604  0.         0.         0.        ]
 [0.06695975 0.         0.         0.        ]
 [0.06912987 0.         0.         0.        ]
 [0.06410049 0.         0.         0.        ]
 [0.06759421 0.         0.         0.        ]
 [0.0698159  0.         0.         0.        ]
 [0.06940781 0.         0.         0.        ]
 [0.06897809 0.         0.         0.        ]]
<NDArray 20x4 @cpu(0)>

In [122]:
from mxnet.gluon import nn 
x=nd.uniform(0,1.0,shape=(2,3))
x,(x<0.4)*x*(1/0.4)  

(
 [[0.6730101  0.08428815 0.80181587]
  [0.95110226 0.18531293 0.03498933]]
 <NDArray 2x3 @cpu(0)>,
 
 [[0.         0.21072038 0.        ]
  [0.         0.46328232 0.08747333]]
 <NDArray 2x3 @cpu(0)>)

# 实现Dropout层

In [2]:
from mxnet import nd 
def dropout(x,drop_pro=0):
    keep_pro=1-drop_pro
    assert 0<=keep_pro<=1
    if keep_pro==0:
        return x.zeros_like() 
    y=nd.uniform(0,1.0,x.shape)<keep_pro
#     保证 期望E(dropout(x))==x
    scale=1/keep_pro
    return x*y*scale

x=nd.normal(scale=1,shape=(2,3))
dropout(x,.4)


[[ 1.9384642   0.806341    0.        ]
 [ 0.25504243 -0.          2.596785  ]]
<NDArray 2x3 @cpu(0)>

In [144]:

c=nd.arange(6).reshape((1,2,3))
from mxnet import image 
c,image.imresize(c,4,4,2)

(
 [[[0. 1. 2.]
   [3. 4. 5.]]]
 <NDArray 1x2x3 @cpu(0)>,
 
 [[[-0.31640625  0.68359375  1.6835938 ]
   [ 0.6796875   1.6796875   2.6796875 ]
   [ 2.3203125   3.3203125   4.3203125 ]
   [ 3.3164062   4.3164062   5.3164062 ]]
 
  [[-0.31640625  0.68359375  1.6835938 ]
   [ 0.6796875   1.6796875   2.6796875 ]
   [ 2.3203125   3.3203125   4.3203125 ]
   [ 3.3164062   4.3164062   5.3164062 ]]
 
  [[-0.31640625  0.68359375  1.6835938 ]
   [ 0.6796875   1.6796875   2.6796875 ]
   [ 2.3203125   3.3203125   4.3203125 ]
   [ 3.3164062   4.3164062   5.3164062 ]]
 
  [[-0.31640625  0.68359375  1.6835938 ]
   [ 0.6796875   1.6796875   2.6796875 ]
   [ 2.3203125   3.3203125   4.3203125 ]
   [ 3.3164062   4.3164062   5.3164062 ]]]
 <NDArray 4x4x3 @cpu(0)>)

In [156]:
def vgg_block(num_convs,channels):
    net=nn.Sequential()
    with net.name_scope():
        for i in range(num_convs):
            net.add(nn.Conv2D(channels,kernel_size=3,padding=1))
        net.add(nn.MaxPool2D(pool_size=2,strides=2))
    return net 
def vgg_stack(convs_channels):
    net=nn.Sequential()
    with net.name_scope():
        for num_convs,channels in convs_channels:
            net.add(vgg_block(num_convs,channels))
    return net 
convs_channels=[[1,64],[2,128],[2,256]]
net=nn.Sequential()
with net.name_scope():
    net.add(vgg_stack(convs_channels),
            nn.AvgPool2D(pool_size=4,strides=2),
           nn.Dense(4096),
           nn.Dense(4096),
           nn.Dropout(.5),
           nn.Dense(10))
net.initialize()
net.collect_params()
# net 

sequential63_ (
  Parameter sequential63_sequential0_sequential0_conv0_weight (shape=(64, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential0_conv0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential1_conv0_weight (shape=(128, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential1_conv0_bias (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential1_conv1_weight (shape=(128, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential1_conv1_bias (shape=(128,), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential2_conv0_weight (shape=(256, 0, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential2_conv0_bias (shape=(256,), dtype=<class 'numpy.float32'>)
  Parameter sequential63_sequential0_sequential2_conv1_weight (shape=(256, 0, 3, 3), dtype=<class 